# Crypto MACD/RSI Indicators Trading

In this paper we studied the possible profit using a simple strategy based on Macd and Rsi Indicators.

We compared the profit achieve with our Macd/Rsi Strategy with a Naive Holding Strategy and a Naive Random Trading Strategy.

Remember that this is not Financial Advice, Invest at your risk.

In [224]:
from stockstats import StockDataFrame
import pandas as pd
import numpy as np
import yfinance as yf
import random

In [225]:
# Calculating the indicators
def make_indicators(df):
    
    df = StockDataFrame.retype(df)
    df['rsi_10']
    df['macd']
    return df.drop(['rs_10', 'close_-1_s', 'close_-1_d'], axis = 1).dropna()

## Algorithm

All strategies use re-invest all profit

* ```strategy(df)``` buy when the macd is greater then zero or when the rsi is less then 30 and sell only in positive if the rsi is greater then 40.

* ```random_baseline(df)``` buy using a coin flip and sell with a profit of 3%

* ```holding_baseline(df)``` buy the first day and sell the last.

In [231]:
# MACD and RSI 10 strategy
def strategy(df):
    
    # We'll buy just with the open value
    prices = df['open']
    # Setting all local variables to zero
    b_price = profit = holding_time = amount = transaction_number = total_holding = p_holding = 0
    # Setting budget
    budget = 100
    
    # For each day
    for i in range(len(df) - 1):
        
        # If we bought then we have to calculate the possible profit, the holding transaction time and the total holding time
        if b_price != 0:
            possible_profit = amount*(prices[i + 1] - b_price)
            holding_time += 1
            total_holding +=1
            
        # Else we just set possible_profit = 0 
        else:
            possible_profit = 0
        
        # If rsi_10 < 30 or macd > 0 and we didn't bought we can buy the coin
        if ((df['rsi_10'][i] < 30 or df['macd'][i] > 0) and b_price == 0):
            
            # The amount is caluclated with the budget/actual price
            amount = budget/prices[i + 1]
            # The buy price is the tomorrow open value
            b_price = prices[i + 1]
            transaction_number += 1
        
        # If we got a positive profit and we bought and rsi_10 > 40 or we hold the coin for more then 10 days then we can sell
        if (df['rsi_10'][i] > 40 or holding_time > 10) and possible_profit > 0.2 and b_price != 0:
            # Setting the b_price to zero so we can buy again if needed
            b_price = 0
            # Setting holding_time to zero so we can count again for selling option
            holding_time = 0
            # Adding the profit to the budget so we can re-invest it
            budget += possible_profit
        
        
    # If we did some transaction  we calculate the average holding days
    if transaction_number != 0:
        p_holding = (total_holding/transaction_number)
        
    print("Total profit for RSI/MACD Strategy: {} %, average holding for transaction: {} days".format((budget - 100), p_holding))
    
    return budget - 100, p_holding

# Naive strategy that buy random
def random_baseline(df):
    
    prices = df['open']
    b_price = profit = holding_time = amount = transaction_number = total_holding = p_holding = 0
    budget = 100
    
    
    for i in range(len(df) - 1):
        
        # Flip the coin
        r = random.randint(0,1)
        
        # If we bought just upgrading the variables
        if b_price != 0:
            possible_profit = amount*(prices[i + 1] - b_price)
            holding_time += 1
            total_holding +=1
        else:
            possible_profit = 0
        
        # If we got the coin flip then we can buy
        if r == 1 and b_price == 0:
            amount = budget/prices[i + 1]
            b_price = prices[i + 1]
            transaction_number += 1
        
        # If we hit the take profit we just sell the coin
        if possible_profit > 3 and b_price != 0:
            b_price = 0
            holding_time = 0
            budget += possible_profit
    
    # If we did at least one transaction we calculate the average holding time
    if transaction_number != 0:
        
        p_holding = (total_holding/transaction_number)
    
            
    print("Total profit for Random Strategy: {} %, average holding for transaction: {} days".format((budget - 100), p_holding))
    return budget - 100, p_holding

# Naive strategy, that buy and hold  
def holding_baseline(df):
    amount = 100/df['open'][0]
    profit = amount*(df['open'][-1] - df['open'][0])
    print("Total profit for Baseline Holding: {} %\n".format(profit))  
    return profit - 100

In [232]:
# Creating sample from dataframe

def single_sample(df, sample_dim):
    s = np.random.choice(df.index[:-sample_dim],1)
    sample = df.iloc[df.index.get_loc(s[0]):df.index.get_loc(s[0])+sample_dim]
    return sample

## Evaluate

We evaluated the possible profit using ten 15 days random samples and make the average of the profit for each strategy

In [242]:
# Evaluating

def evaluate(df, sample_number):
    t_s_holding = t_r_holding = total_s = total_r = total_h = 0
    for i in range(sample_number):
        
        sample = single_sample(df, 15)
        df = pd.concat([df, sample]).drop_duplicates(keep=False)
        
        s, s_holding = strategy(sample)

        r, r_holding = random_baseline(sample)

        h = holding_baseline(sample)
        
        # Sum all profit 
        total_s += s
        total_r += r
        total_h += h
        
        # Sum all holding time
        t_s_holding += s_holding
        t_r_holding += r_holding
        
        
    
    print("\n\n Average Profit for RSI/MACD Strategy: {}, Average Holding {} days".format((total_s/sample_number), t_s_holding))
    print("\n\n Average Profit for Random Strategy: {}, Average Holding {} days".format((total_r/sample_number), t_r_holding))
    print("\n\n Average Profit for Holding: {}".format((total_h/sample_number)))

## Evaluate different coins

If you need to evaluate different coins just change the value of ```coin``` with the one that you want to study

In [234]:
coin = "BTC-EUR"

df = yf.Ticker(coin).history(period="max").drop(['Dividends', 'Stock Splits'],axis = 1)
df = StockDataFrame.retype(df)
df = make_indicators(df)

## Result

With a timeframe of 15 days our strategy beat both the Baseline!


Average Profit for RSI/MACD Strategy: 10.321033323478247, Average Holding 51.3 days


Average Profit for Random Strategy: 6.532289690381719, Average Holding 65.33333333333334 days


Average Profit for Holding: -92.64596349000122

## Conclusion

With a short term timeframe the RSI/MACD Strategy beat both the Baseline, but with a long term the Holding Strategy is more efficient.

That means that we didn't beat the Market :(